In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=baee2349478661f9ac89501ff5f74cd8788d052798a3534b735a6cec4ecc0bcc
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [15]:
from pyspark.sql import SparkSession

# Créer une session Spark
spark = SparkSession.builder \
    .appName("PlanetDiscoveryAnalysis") \
    .getOrCreate()

# Chemin vers le fichier CSV
file_path = "planets_dataset.csv"

# Lecture du fichier CSV dans un DataFrame Spark
planets_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Afficher le schéma des données pour vérifier les types de colonnes
planets_df.printSchema()


root
 |-- Name: string (nullable = true)
 |-- Num_Moons: integer (nullable = true)
 |-- Minerals: integer (nullable = true)
 |-- Gravity: double (nullable = true)
 |-- Sunlight_Hours: double (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Rotation_Time: double (nullable = true)
 |-- Water_Presence: integer (nullable = true)
 |-- Colonisable: integer (nullable = true)



In [16]:
# Calculer la masse moyenne des planètes
mass_avg = planets_df.agg({"Gravity": "avg"}).alias("Masse Moyenne")
mass_avg.show()


+------------------+
|      avg(Gravity)|
+------------------+
|1.5475425398260738|
+------------------+



In [17]:
# Distribution du nombre de lunes des planètes
moons_distribution = planets_df.groupBy("Num_Moons").count()
moons_distribution.show()


+---------+-----+
|Num_Moons|count|
+---------+-----+
|        1|10045|
|        6| 9939|
|        3|10008|
|        5|10073|
|        9| 9828|
|        4|10059|
|        8| 9925|
|        7|10119|
|        2|10171|
|        0| 9833|
+---------+-----+



In [18]:
from pyspark.ml.feature import VectorAssembler

# Assembler les colonnes de caractéristiques
assembler = VectorAssembler(
    inputCols=["Gravity", "Num_Moons", "Sunlight_Hours", "Temperature", "Rotation_Time"],
    outputCol="features"
)

# Appliquer l'assemblage pour créer les features
ml_data = assembler.transform(planets_df)

# Sélectionner les caractéristiques et la cible (Colonisable)
ml_data = ml_data.select("features", "Colonisable")


In [20]:
from pyspark.sql.functions import col, when

# Convertir la colonne 'Colonisable' en entier (1 pour True, 0 pour False)
ml_data = ml_data.withColumn("Colonisable", when(col("Colonisable") == 1.0, 1).otherwise(0))


In [21]:
from pyspark.ml.classification import LogisticRegression

# Initialiser le modèle de régression logistique
lr = LogisticRegression(featuresCol='features', labelCol='Colonisable')

# Entraîner le modèle
model = lr.fit(ml_data)

# Faire des prédictions
predictions = model.transform(ml_data)
predictions.select("features", "Colonisable", "prediction").show()


+--------------------+-----------+----------+
|            features|Colonisable|prediction|
+--------------------+-----------+----------+
|[1.98160285946924...|          0|       0.0|
|[1.38815048308067...|          0|       0.0|
|[2.53082672515200...|          0|       0.0|
|[0.27219740413253...|          0|       0.0|
|[1.73105991307994...|          0|       0.0|
|[2.32248535362167...|          0|       0.0|
|[1.27298319329060...|          0|       0.0|
|[0.56606721098957...|          0|       0.0|
|[0.84005516112389...|          0|       0.0|
|[2.40329834626856...|          0|       0.0|
|[1.60799048076341...|          0|       0.0|
|[0.53131129377391...|          0|       0.0|
|[1.82827629773893...|          0|       0.0|
|[0.73466116785639...|          0|       0.0|
|[1.25695583878599...|          0|       0.0|
|[1.17737973236620...|          0|       0.0|
|[0.34418261967006...|          0|       0.0|
|[1.19896489552282...|          0|       0.0|
|[0.48054407242641...|          0|

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Évaluer la précision du modèle
evaluator = MulticlassClassificationEvaluator(labelCol="Colonisable", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Précision du modèle : {accuracy}")


Précision du modèle : 0.9441


In [23]:
# Compter les colonisables et non colonisables
ml_data.groupBy("Colonisable").count().show()


+-----------+-----+
|Colonisable|count|
+-----------+-----+
|          1| 5590|
|          0|94410|
+-----------+-----+



In [24]:
# Diviser les données en un ensemble d'entraînement et un ensemble de test (70% entraînement, 30% test)
train_data, test_data = ml_data.randomSplit([0.7, 0.3], seed=42)

# Entraîner le modèle sur les données d'entraînement
model = lr.fit(train_data)

# Faire des prédictions sur les données de test
test_predictions = model.transform(test_data)

# Évaluer la précision sur l'ensemble de test
test_accuracy = evaluator.evaluate(test_predictions)
print(f"Précision du modèle sur les données de test : {test_accuracy}")


Précision du modèle sur les données de test : 0.9437673222693425


In [25]:
# Évaluer d'autres métriques : F1-score, précision et rappel
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="Colonisable", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(test_predictions)
print(f"F1-Score du modèle : {f1_score}")


F1-Score du modèle : 0.9164643816972486


In [26]:
from pyspark.ml.classification import DecisionTreeClassifier

# Initialiser un modèle d'arbre de décision
dt = DecisionTreeClassifier(featuresCol="features", labelCol="Colonisable")

# Entraîner le modèle
dt_model = dt.fit(ml_data)

# Faire des prédictions
dt_predictions = dt_model.transform(ml_data)
dt_predictions.select("features", "Colonisable", "prediction").show()


+--------------------+-----------+----------+
|            features|Colonisable|prediction|
+--------------------+-----------+----------+
|[1.98160285946924...|          0|       0.0|
|[1.38815048308067...|          0|       0.0|
|[2.53082672515200...|          0|       0.0|
|[0.27219740413253...|          0|       0.0|
|[1.73105991307994...|          0|       0.0|
|[2.32248535362167...|          0|       0.0|
|[1.27298319329060...|          0|       0.0|
|[0.56606721098957...|          0|       0.0|
|[0.84005516112389...|          0|       0.0|
|[2.40329834626856...|          0|       0.0|
|[1.60799048076341...|          0|       0.0|
|[0.53131129377391...|          0|       0.0|
|[1.82827629773893...|          0|       0.0|
|[0.73466116785639...|          0|       0.0|
|[1.25695583878599...|          0|       0.0|
|[1.17737973236620...|          0|       0.0|
|[0.34418261967006...|          0|       0.0|
|[1.19896489552282...|          0|       0.0|
|[0.48054407242641...|          0|

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Évaluer la précision du modèle
evaluator = MulticlassClassificationEvaluator(labelCol="Colonisable", predictionCol="prediction", metricName="accuracy")
dt_accuracy = evaluator.evaluate(dt_predictions)
print(f"Précision de l'Arbre de Décision : {dt_accuracy}")


Précision de l'Arbre de Décision : 0.9441


In [28]:
from pyspark.ml.classification import RandomForestClassifier

# Initialiser le modèle de Forêt Aléatoire
rf = RandomForestClassifier(featuresCol="features", labelCol="Colonisable", numTrees=10)

# Entraîner le modèle
rf_model = rf.fit(ml_data)

# Faire des prédictions
rf_predictions = rf_model.transform(ml_data)
rf_predictions.select("features", "Colonisable", "prediction").show()


+--------------------+-----------+----------+
|            features|Colonisable|prediction|
+--------------------+-----------+----------+
|[1.98160285946924...|          0|       0.0|
|[1.38815048308067...|          0|       0.0|
|[2.53082672515200...|          0|       0.0|
|[0.27219740413253...|          0|       0.0|
|[1.73105991307994...|          0|       0.0|
|[2.32248535362167...|          0|       0.0|
|[1.27298319329060...|          0|       0.0|
|[0.56606721098957...|          0|       0.0|
|[0.84005516112389...|          0|       0.0|
|[2.40329834626856...|          0|       0.0|
|[1.60799048076341...|          0|       0.0|
|[0.53131129377391...|          0|       0.0|
|[1.82827629773893...|          0|       0.0|
|[0.73466116785639...|          0|       0.0|
|[1.25695583878599...|          0|       0.0|
|[1.17737973236620...|          0|       0.0|
|[0.34418261967006...|          0|       0.0|
|[1.19896489552282...|          0|       0.0|
|[0.48054407242641...|          0|

In [29]:
# Évaluer la précision du modèle
rf_accuracy = evaluator.evaluate(rf_predictions)
print(f"Précision de la Forêt Aléatoire : {rf_accuracy}")


Précision de la Forêt Aléatoire : 0.9441


In [30]:
from pyspark.ml.regression import LinearRegression

# Assembler les colonnes de caractéristiques pour la régression (exclure la cible "Colonisable")
assembler = VectorAssembler(
    inputCols=["Num_Moons", "Minerals", "Sunlight_Hours", "Temperature", "Rotation_Time", "Water_Presence"],
    outputCol="features"
)
ml_data_reg = assembler.transform(planets_df)

# Entraîner un modèle de régression linéaire pour prédire la gravité
lr = LinearRegression(featuresCol="features", labelCol="Gravity")
lr_model = lr.fit(ml_data_reg)

# Faire des prédictions
lr_predictions = lr_model.transform(ml_data_reg)
lr_predictions.select("features", "Gravity", "prediction").show()


+--------------------+-------------------+------------------+
|            features|            Gravity|        prediction|
+--------------------+-------------------+------------------+
|[5.0,59.0,5.81681...|  1.981602859469247|1.5517093902228312|
|[8.0,672.0,14.715...| 1.3881504830806715|1.5492688159240475|
|[3.0,764.0,22.902...| 2.5308267251520093| 1.547190240209926|
|[4.0,881.0,1.1764...|  0.272197404132533|1.5451714063273365|
|[7.0,1003.0,21.48...| 1.7310599130799436|1.5490842269264082|
|[4.0,900.0,13.416...| 2.3224853536216754| 1.546145594424239|
|[1.0,877.0,5.6888...| 1.2729831932906077|1.5441437138902177|
|[7.0,515.0,9.3069...| 0.5660672109895787| 1.549538246155466|
|[3.0,743.0,0.6059...| 0.8400551611238913| 1.544879194274711|
|[0.0,576.0,18.218...| 2.4032983462685635|1.5483533540488346|
|[0.0,417.0,6.7932...| 1.6079904807634198|1.5494070834346465|
|[1.0,751.0,19.917...| 0.5313112937739153|1.5464127294091257|
|[9.0,879.0,13.556...|  1.828276297738934|1.5464945687582978|
|[2.0,60

In [31]:
from pyspark.ml.evaluation import RegressionEvaluator

# Évaluer la précision du modèle
evaluator = RegressionEvaluator(labelCol="Gravity", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(lr_predictions)
print(f"Erreur quadratique moyenne (RMSE) du modèle de régression linéaire : {rmse}")


Erreur quadratique moyenne (RMSE) du modèle de régression linéaire : 0.8381086402387488


In [32]:
from pyspark.ml.classification import LinearSVC

# Initialiser le modèle SVM linéaire
svm = LinearSVC(featuresCol="features", labelCol="Colonisable")

# Entraîner le modèle SVM
svm_model = svm.fit(ml_data)

# Faire des prédictions
svm_predictions = svm_model.transform(ml_data)

# Afficher les prédictions
svm_predictions.select("features", "Colonisable", "prediction").show()


+--------------------+-----------+----------+
|            features|Colonisable|prediction|
+--------------------+-----------+----------+
|[1.98160285946924...|          0|       0.0|
|[1.38815048308067...|          0|       0.0|
|[2.53082672515200...|          0|       0.0|
|[0.27219740413253...|          0|       0.0|
|[1.73105991307994...|          0|       0.0|
|[2.32248535362167...|          0|       0.0|
|[1.27298319329060...|          0|       0.0|
|[0.56606721098957...|          0|       0.0|
|[0.84005516112389...|          0|       0.0|
|[2.40329834626856...|          0|       0.0|
|[1.60799048076341...|          0|       0.0|
|[0.53131129377391...|          0|       0.0|
|[1.82827629773893...|          0|       0.0|
|[0.73466116785639...|          0|       0.0|
|[1.25695583878599...|          0|       0.0|
|[1.17737973236620...|          0|       0.0|
|[0.34418261967006...|          0|       0.0|
|[1.19896489552282...|          0|       0.0|
|[0.48054407242641...|          0|

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialiser un évaluateur de classification multiclasse
evaluator = MulticlassClassificationEvaluator(labelCol="Colonisable", predictionCol="prediction", metricName="accuracy")

# Évaluer la précision du modèle SVM
svm_accuracy = evaluator.evaluate(svm_predictions)
print(f"Précision du modèle SVM : {svm_accuracy}")


Précision du modèle SVM : 0.9441


In [35]:
from pyspark.ml.classification import GBTClassifier

# Initialiser un modèle d'arbres boostés par gradient
gbt = GBTClassifier(featuresCol="features", labelCol="Colonisable", maxIter=10)

# Entraîner le modèle
gbt_model = gbt.fit(ml_data)

# Faire des prédictions
gbt_predictions = gbt_model.transform(ml_data)

# Afficher les prédictions
gbt_predictions.select("features", "Colonisable", "prediction").show()


+--------------------+-----------+----------+
|            features|Colonisable|prediction|
+--------------------+-----------+----------+
|[1.98160285946924...|          0|       0.0|
|[1.38815048308067...|          0|       0.0|
|[2.53082672515200...|          0|       0.0|
|[0.27219740413253...|          0|       0.0|
|[1.73105991307994...|          0|       0.0|
|[2.32248535362167...|          0|       0.0|
|[1.27298319329060...|          0|       0.0|
|[0.56606721098957...|          0|       0.0|
|[0.84005516112389...|          0|       0.0|
|[2.40329834626856...|          0|       0.0|
|[1.60799048076341...|          0|       0.0|
|[0.53131129377391...|          0|       0.0|
|[1.82827629773893...|          0|       0.0|
|[0.73466116785639...|          0|       0.0|
|[1.25695583878599...|          0|       0.0|
|[1.17737973236620...|          0|       0.0|
|[0.34418261967006...|          0|       0.0|
|[1.19896489552282...|          0|       0.0|
|[0.48054407242641...|          0|

In [36]:
# Évaluer le modèle GBT avec la courbe ROC
gbt_auc = evaluator.evaluate(gbt_predictions)
print(f"Aire sous la courbe ROC (GBT) : {gbt_auc}")


Aire sous la courbe ROC (GBT) : 0.9441
